In [26]:
#!pip install yfinance

# Invest Portfolio

Ivan has a large amout of money, and he starts to think how to invest this money to gain maximum profit.
He chooses 10 candidate companies for that, but since Ivan is new in investing he needs your advice. In this notebook we will help Ivan in his journey.      

In [27]:
# 10 companies to consider, key - ticker name, value -  short description.
tickers = {
    "BA": "The Boeing Company",
    "MCD": "McDonald's Corporation",
    "GOOGL": "Google Company",
    "DIS": "The Walt Disney Company",
    "PFE": "Pfizer Inc.",
    "AMT": "American Tower Corporation",
    "VOD": "Vodafone Group Public Limited Company",
    "BNP.PA": "BNP Paribas Bank",
    "ADS.DE": "Adidas",
    "TM": "Toyota Motor Corporation",
}


## Task 1 Download and preprocess data

We will use data from 2010 till 2021 for training and data from 2021 till 2024  for testing purpose.



In [28]:
start_train_date = "2010-01-01"
end_train_date = "2021-01-01"
start_test_date = "2021-01-01"
end_test_date = "2024-01-01"

Write a function that takes ticker name, start and end dates and returns pandas series of annual stock prices.



In [29]:
import yfinance as yf
import pandas as pd
import numpy as np


def download_annual_stock_prices(tickers_list, buy_date="2010-01-01", sell_date="2021-01-01"):
    """Download the annual stock price using yf.download"""
    return yf.download(tickers_list, start=buy_date, end=sell_date, interval="1mo", progress=False)["Open"][::12]

In [30]:
annual_stock_price_df = download_annual_stock_prices(list(tickers), start_train_date, end_train_date)
test_annual_stock_price_df = download_annual_stock_prices(list(tickers), start_test_date, end_test_date)

In [31]:
test_annual_stock_price_df

Ticker,ADS.DE,AMT,BA,BNP.PA,DIS,GOOGL,MCD,PFE,TM,VOD
Date,,,,,,,,,,
2021-01-01,297.899994,226.070007,210.000000,43.105000,182.259995,88.000000,214.490005,36.869999,155.809998,17.040001
2022-01-01,254.350006,292.369995,204.000000,61.110001,155.830002,145.054993,269.489990,58.500000,186.809998,15.200000
2023-01-01,128.339996,214.000000,192.949997,53.910000,88.980003,89.589996,263.529999,51.009998,137.960007,10.290000


In [32]:
annual_stock_price_df

Ticker,ADS.DE,AMT,BA,BNP.PA,DIS,GOOGL,MCD,PFE,TM,VOD
Date,,,,,,,,,,
2010-01-01,37.770000,43.490002,55.720001,55.900002,32.500000,15.689439,62.630001,17.333965,84.750000,23.649338
2011-01-01,49.279999,51.910000,66.150002,48.299999,37.740002,14.926927,77.099998,16.793169,79.019997,26.941896
2012-01-01,49.509998,60.450001,74.699997,30.450001,37.970001,16.339840,101.330002,20.740038,67.389999,28.695210
2013-01-01,67.330002,78.389999,76.550003,42.584999,50.799999,18.003504,89.400002,24.222012,94.779999,26.258919
2014-01-01,92.639999,79.510002,136.009995,56.650002,76.040001,27.914413,96.809998,28.908918,121.599998,39.755352
2015-01-01,57.619999,99.160004,131.070007,49.259998,94.910004,26.629999,94.129997,29.667933,126.230003,34.080002
2016-01-01,89.910004,96.290001,141.380005,52.590000,103.120003,38.110001,117.250000,30.218216,121.519997,32.119999
2017-01-01,149.750000,106.730003,156.300003,60.119999,105.300003,40.030998,121.860001,31.024668,118.169998,24.840000
2018-01-01,167.149994,143.160004,295.750000,62.250000,108.949997,52.651001,173.729996,34.516129,127.430000,32.160000



Let's define $\xi_i$ as random variable equals annual return ratio of the $\text{stock}_i$  relative to the buying price.

Convert annual stock price to annual return ratio considering that we've bought stock at moment $t=0$.
i.e. if the annoaul stock prices are $(10, 15, 12, 13)$,
 annual return ratio will be $(0.5, -0.3, 0.1)$

In [33]:
def get_annual_return_ratio_from_initial(prices_df):
    diff_df = prices_df.diff(1).dropna()
    return diff_df.div(prices_df.iloc[0])


test_stock_price_df = pd.DataFrame({
    "ticker_1": [1, 3, 2, 5],
    "ticker_2": [10, 15, 12, 13],
})
test_annual_return_ratio_df = pd.DataFrame({
    "ticker_1": [2.0, -1.0, 3.0],
    "ticker_2": [0.5, -0.3, 0.1],
})
result_np = get_annual_return_ratio_from_initial(test_stock_price_df).values
np.testing.assert_array_equal(result_np, test_annual_return_ratio_df.values)

In [34]:
annual_return_ratio_df = get_annual_return_ratio_from_initial(annual_stock_price_df)
test_annual_return_ratio_df = get_annual_return_ratio_from_initial(test_annual_stock_price_df)

## Task 2 Simple approach

Let's invest all money in most profitable company but for that we need to calculate $E\xi_i$ first.

We can calculate this value from company annual return ratio and from stock price. Let's do both to make sure that we do everything right.

In [35]:
eps = 1e-7


def get_expectation_ratio_from_return_ratio(ticker_return_ratio_df):
    return ticker_return_ratio_df.mean()


def get_expectation_ratio_from_stock_price(ticker_stock_price_df):
    pct_price_change_df = (ticker_stock_price_df.iloc[-1] / ticker_stock_price_df.iloc[0]) - 1
    return pct_price_change_df / (len(ticker_stock_price_df) - 1)

In [36]:
expectation_ratio_1 = get_expectation_ratio_from_return_ratio(annual_return_ratio_df)
expectation_ratio_2 = get_expectation_ratio_from_stock_price(annual_stock_price_df)
assert sum(abs(expectation_ratio_1 - expectation_ratio_2)) < eps

Looks like we are ready to implement our greedy approach:

$$ \text{company-to-invest} = \max_i E \xi_i $$

Let's implement this approach having tickers and dates of buying and selling:

In [37]:
def get_expectation_ratio_for_greedy_performance(prices_df):
    return get_expectation_ratio_from_stock_price(prices_df)


def find_ticker_for_greedy_selection(prices_df):
    expectation_df = get_expectation_ratio_from_stock_price(prices_df)
    return expectation_df.idxmax()

In [38]:
aapl_df = download_annual_stock_prices(["AAPL"], start_train_date, end_train_date)
assert 0.8 < get_expectation_ratio_from_stock_price(aapl_df) < 0.9

In [39]:
best_greedy_ticker = find_ticker_for_greedy_selection(annual_stock_price_df)
greedy_ticker_train_performance = get_expectation_ratio_for_greedy_performance(
    annual_stock_price_df[best_greedy_ticker])
greedy_ticker_test_performance = get_expectation_ratio_for_greedy_performance(
    test_annual_stock_price_df[best_greedy_ticker])
print(f"\n ticker to invest in greedy approach: {best_greedy_ticker}")
print(f"Train expectation ratio: {greedy_ticker_train_performance}")
print(f"Test expectation ratio: {greedy_ticker_test_performance}")


 ticker to invest in greedy approach: ADS.DE
Train expectation ratio: 0.6672755739493247
Test expectation ratio: -0.2845921467482695


Looks like our greedy method is not very helpful.

## Task 3 Variance

If stock has big variance it means that investor cannot be sure about the future. What is the variance in our case?

Your task is to find $D\xi_i$

In [40]:
def get_variance_from_return_ratio(ticker_return_ratio_df):
    return ticker_return_ratio_df.var(ddof=0)


def get_ticker_variance(prices_df):
    return_ratio = get_annual_return_ratio_from_initial(prices_df)
    return get_variance_from_return_ratio(return_ratio)

In [41]:
aapl_df = download_annual_stock_prices(["AAPL"], start_train_date, end_train_date)
assert 1.9 < get_ticker_variance(aapl_df) < 2.0

Then let's reformulate our task, we will select stock to invest by maximizing the following:

$$\text{company-to-invest} = \max_i \left( \frac{E\xi_i} {\sqrt{ D\xi_i}} \right)$$

In [42]:
def calc_expectation_ratio_divided_by_variance(prices_df):
    expectation_df = get_expectation_ratio_from_stock_price(prices_df)
    variance_df = get_ticker_variance(prices_df)
    return expectation_df.div(variance_df.pow(0.5))


def find_ticker_for_max_expectation_divided_by_var(prices_df):
    return calc_expectation_ratio_divided_by_variance(prices_df).idxmax()

In [43]:
best_expectation_var_ticker = find_ticker_for_max_expectation_divided_by_var(annual_stock_price_df)
expectation_var_train_performance = get_expectation_ratio_for_greedy_performance(
    annual_stock_price_df[best_expectation_var_ticker])
greedy_ticker_test_performance = get_expectation_ratio_for_greedy_performance(
    test_annual_stock_price_df[best_expectation_var_ticker])

print(f"\n ticker to invest in expectation-variance approach: {best_expectation_var_ticker}")
print(f"Train expectation ratio: {expectation_var_train_performance}")
print(f"Test expectation ratio: {greedy_ticker_test_performance}")


 ticker to invest in expectation-variance approach: DIS
Train expectation ratio: 0.347046133188101
Test expectation ratio: -0.25589815088687423


Still not good...

## Task 4 Linear combination

In investing, there is a golden rule not to put all your eggs in one basket. So why shouldn't Ivan diversify his investments?

Let's define the weight vector $w = (w_1, \ldots, w_n)$ such that $|w| = \sum_{i} w_i = 1;$ and  $ 0 \le w_i \le 1$ .

Our solution can then be represented as:
$\xi = \sum_i w_i \xi_i$

**Task:** Expand the final formula

$$ \max_w \left (  \frac{E[\xi]} { \sqrt{D[\xi]} } \right)   =
    \max_w \left (  \frac{E \left[ \sum_i w_i \xi_i \right] } { \sqrt{D \left[ \sum_i w_i \xi_i \right] } } \right)  = \ldots
$$  



To find best vector $w$ we have to create a function that returns our metric having vector $w$

In [44]:
def get_expectation(prices_df, weights):
    """Calculate expected profit if we invest in each ticker with certain weight."""
    expectation_df = get_expectation_ratio_from_stock_price(prices_df)
    return expectation_df.mul(weights).sum()


def create_functional(priced_df):
    annual_ratio_df: pd.DataFrame = get_annual_return_ratio_from_initial(priced_df)

    def wrapper(weights):
        weighted_annual_ratio_sum_df = annual_ratio_df.mul(weights).sum(axis=1)  # sum(w_i * xi_i)

        weighted_expectation = weighted_annual_ratio_sum_df.mean()
        weighted_variance = weighted_annual_ratio_sum_df.var(ddof=0)
        return weighted_expectation / np.sqrt(weighted_variance)

    return wrapper


functional = create_functional(annual_stock_price_df)

equal_weights = np.array([1 / len(tickers) for _ in tickers])
metric_for_equal_weights = functional(equal_weights)

assert 1.0 < metric_for_equal_weights < 1.5

In [45]:
metric_for_equal_weights

1.0415405684382606

## Task 5 Optimization

But how to find best weights?

Let's define optimization problem and use _minimize_ function from _sicpy_ package.

**Your task** Find best weights and evaluate train and test results.

In [46]:
from scipy.optimize import minimize


def constraint(x):
    return sum(x) - 1


con = {"type": "eq", "fun": constraint}
bounds = [(0, 1) for _ in range(len(tickers))]
minimization_functional = lambda weights: - functional(weights)

task_5_result = minimize(minimization_functional, equal_weights, constraints=con, bounds=bounds)
task_5_result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -2.2500799473470083
       x: [ 3.112e-02  2.954e-02  2.538e-14  1.704e-13  1.975e-01
            1.620e-13  1.565e-01  5.854e-01  1.891e-13  1.870e-13]
     nit: 12
     jac: [ 5.783e-03  1.620e-03  5.683e+00  1.792e+00  5.433e-04
            2.848e+00  1.668e-03 -1.018e-03  1.797e+00  3.927e+00]
    nfev: 141
    njev: 12

## Task 6 Bonus: Experiment with initial weights


For running mimimzation algorithm we need to define initial vector $w_0$, but how to do that?

One of the approaches is to try $N$ different starting points and select the best.

**Your task**
1. Implement algorithm that generates $N$ random vectors for weights then tries to minimize functional $N$ times using these vectors as a starting points. And returns the best founded weights.
2. Apply this  algorithm for our task and compare results


In [47]:
from tqdm import tqdm


def generate_random_weights(vector_size):
    weights = np.random.rand(vector_size)
    weights /= np.sum(weights)
    return weights


def random_search_for_initial_weights(functional_to_minimize, n_tries, vector_size):
    best_func_value = np.inf
    best_weights = None
    jac = None
    for _ in tqdm(range(n_tries)):
        weights = generate_random_weights(vector_size)
        res = minimize(functional_to_minimize, weights, constraints=con, bounds=bounds)
        if res.fun < best_func_value:
            best_func_value = res.fun
            best_weights = res.x
            jac = res.jac

    return best_weights, best_func_value, -jac


optimal_weights, func_value, func_jac = random_search_for_initial_weights(minimization_functional, 100, vector_size=len(tickers))

print("Optimal weights:", optimal_weights.round(4))
print("Maximized F(weights):", func_value)
print("train expectation:", get_expectation(annual_stock_price_df, optimal_weights))
print("test expectation:", get_expectation(test_annual_stock_price_df, optimal_weights))

test_task_5_expectation = get_expectation(test_annual_stock_price_df, task_5_result.x)
test_task_6_expectation = get_expectation(test_annual_stock_price_df, optimal_weights)
print("Difference between task 5 and 6:", (test_task_6_expectation - test_task_5_expectation) * 100)

100%|██████████| 100/100 [00:11<00:00,  8.47it/s]

Optimal weights: [0.0311 0.0296 0.     0.     0.1975 0.     0.1564 0.5855 0.     0.    ]
Maximized F(weights): -2.2500800953791757
train expectation: 0.2031289831605738
test expectation: 0.06998188633008273
Difference between task 5 and 6: 0.0012108765645002517


In [48]:
train_df = calc_expectation_ratio_divided_by_variance(annual_stock_price_df).to_frame(name="train").T
test_df = calc_expectation_ratio_divided_by_variance(test_annual_stock_price_df).to_frame(name="test").T
weights_df = pd.DataFrame(optimal_weights, index=annual_stock_price_df.keys(), columns=["weights"]).T
jac_df = pd.DataFrame(func_jac, index=annual_stock_price_df.keys(), columns=["jac"]).T
pd.concat([train_df, test_df, weights_df, jac_df]).astype(np.float16)

Ticker,ADS.DE,AMT,BA,BNP.PA,DIS,GOOGL,MCD,PFE,TM,VOD
train,0.708496,0.898926,0.668457,-0.024704,0.959473,0.826172,0.791016,0.714844,0.372803,-0.059875
test,-2.056641,-0.083435,-3.376953,0.428711,-2.308594,0.014130,0.804688,0.485596,-0.223511,-2.199219
weights,0.031097,0.029556,0.000000,0.000000,0.197510,0.000000,0.156372,0.585449,0.000000,0.000000
jac,0.004452,0.005733,-5.679688,-1.791992,-0.000887,-2.845703,0.000767,-0.000430,-1.796875,-3.929688


In [49]:
test_annual_stock_price_df

Ticker,ADS.DE,AMT,BA,BNP.PA,DIS,GOOGL,MCD,PFE,TM,VOD
Date,,,,,,,,,,
2021-01-01,297.899994,226.070007,210.000000,43.105000,182.259995,88.000000,214.490005,36.869999,155.809998,17.040001
2022-01-01,254.350006,292.369995,204.000000,61.110001,155.830002,145.054993,269.489990,58.500000,186.809998,15.200000
2023-01-01,128.339996,214.000000,192.949997,53.910000,88.980003,89.589996,263.529999,51.009998,137.960007,10.290000


$$ 
    \max_w \left (  \frac{E[\xi]} { \sqrt{D[\xi]} } \right)   =
    \max_w \left (  \frac{E \left[ \sum_i w_i \xi_i \right] } { \sqrt{D \left[ \sum_i w_i \xi_i \right] } } \right)  = 
    \max_w \left ( \frac{\sum_i w_i E_{\xi_i} } { \sqrt{\sum_i w_i^2 D_{\xi_i} + 2 \sum_{i<j} w_i w_j \cdot \text{cov}\left(\xi_i, \xi_j \right)} } \right)
$$  


In [50]:
def create_functional_2(priced_df: pd.DataFrame):
    annual_ratio_df: pd.DataFrame = get_annual_return_ratio_from_initial(priced_df)

    def wrapper(weights: np.ndarray):
        numerator = (weights * annual_ratio_df.mean()).sum()

        denominator_var = ((weights ** 2) * annual_ratio_df.var(ddof=0)).sum()

        cov_df = annual_ratio_df.cov(ddof=0)
        denominator_cov = 0
        for i in range(len(weights)):
            for j in range(i + 1, len(weights)):
                denominator_cov += weights[i] * weights[j] * cov_df.iloc[i, j]

        denominator = np.sqrt(denominator_var + 2 * denominator_cov)
        return numerator / denominator

    return wrapper


functional_1 = create_functional(annual_stock_price_df)
print("functional_1:", functional_1(optimal_weights))

functional_2 = create_functional_2(annual_stock_price_df)
print("functional_2:", functional_2(optimal_weights))


functional_1: 2.2500800953791757
functional_2: 2.250080095379176
